In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
from datetime import timedelta
%matplotlib inline

#from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score

import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

from imblearn.over_sampling import RandomOverSampler


In [2]:
labels = pd.read_csv("../../data/labels_training_set.csv")
df = pd.read_csv("../../data/events_up_to_01062018.csv")

C:\Users\seba\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_persons = pd.read_csv("../../data/trocafone_kaggle_test.csv")

In [4]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["month"] = df["timestamp"].dt.month
df["day"] = df["timestamp"].dt.day

In [5]:
todas_las_personas = df[["person"]].drop_duplicates(subset="person")

# Cantidad de veces que realiza un evento dado en el mes 5

In [6]:
eventos_mes_cinco = df.loc[df["month"] == 5]
cant_por_evento_mes_cinco = pd.concat([pd.get_dummies(eventos_mes_cinco['event']),eventos_mes_cinco[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()

cant_por_evento_mes_cinco.columns = ['person', 'ad campaign hit mes 5', 'brand listing mes 5', 'checkout mes 5',\
                                     'conversion mes 5', 'generic listing mes 5', 'lead mes 5', 'search engine hit mes 5',\
                                     'searched products mes 5', 'staticpage mes 5', 'viewed product mes 5', 'visited site mes 5' ]

# Relleno con ceros las personas que no entraron en el mes 5
cant_por_evento_mes_cinco = pd.merge(cant_por_evento_mes_cinco, todas_las_personas, on="person", how='right').fillna(0)

df_train = pd.merge(cant_por_evento_mes_cinco, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product mes 5', 'visited site mes 5', 'staticpage mes 5', 'lead mes 5'])
cant_por_evento_mes_cinco.head()

,person,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,lead mes 5,search engine hit mes 5,searched products mes 5,staticpage mes 5,viewed product mes 5,visited site mes 5
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
2,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
3,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0
4,000e4d9e,19.0,17.0,1.0,0.0,17.0,0.0,5.0,0.0,0.0,339.0,13.0


In [7]:
#veo cuantos users pierdo
cant_por_evento_predecir = pd.merge(cant_por_evento_mes_cinco, df_persons, on="person", how="inner")
cant_por_evento_predecir["person"].count() - df_persons["person"].count()

0

# Distancia entre ultima entrada al sitio y el ultimo dia que tenemos

In [8]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia_de_todos = timestamp_df["timestamp"].max()
ultima_entrada = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
ultima_entrada["dias_hasta_ultimo"] = (ultimo_dia_de_todos - ultima_entrada["ultimo"]).dt.days
ultima_entrada = ultima_entrada[["person", "dias_hasta_ultimo"]]
ultima_entrada.head()

,person,dias_hasta_ultimo
0,0008ed71,14
1,00091926,0
2,00091a7a,66
3,000ba417,5
4,000c79fe,2


In [9]:
# Veo cuantas personas a predecir pierdo
df_predecir = pd.merge(ultima_entrada, df_persons, on="person", how="inner")
df_predecir["person"].count() - df_persons["person"].count()

0

# Cantidad de veces que realiza un evento dado en el mes 4

In [10]:
eventos_mes_cuatro = df.loc[df["month"] == 4]
cant_por_evento_mes_cuatro = pd.concat([pd.get_dummies(eventos_mes_cuatro['event']),eventos_mes_cuatro[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()

cant_por_evento_mes_cuatro.columns = ['person', 'ad campaign hit mes 4', 'brand listing mes 4', 'checkout mes 4',\
                                     'conversion mes 4', 'generic listing mes 4', 'lead mes 4', 'search engine hit mes 4',\
                                     'searched products mes 4', 'staticpage mes 4', 'viewed product mes 4', 'visited site mes 4' ]

# Relleno con ceros las personas que no entraron en el mes 5
cant_por_evento_mes_cuatro = pd.merge(cant_por_evento_mes_cuatro, todas_las_personas, on="person", how='right').fillna(0)

df_train = pd.merge(cant_por_evento_mes_cuatro, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product mes 4', 'visited site mes 4', 'staticpage mes 4', 'lead mes 4'])
cant_por_evento_mes_cuatro.head()

,person,ad campaign hit mes 4,brand listing mes 4,checkout mes 4,conversion mes 4,generic listing mes 4,lead mes 4,search engine hit mes 4,searched products mes 4,staticpage mes 4,viewed product mes 4,visited site mes 4
0,0019e639,1.0,18.0,2.0,0.0,1.0,0.0,1.0,7.0,0.0,38.0,1.0
1,001ca5ee,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,9.0
2,001dfc31,1.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,2.0
3,001e9aea,10.0,4.0,3.0,0.0,0.0,0.0,10.0,0.0,0.0,15.0,4.0
4,002aea56,1.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,12.0,4.0


In [11]:
#veo cuantos users pierdo
predecir = pd.merge(cant_por_evento_mes_cuatro, df_persons, on="person", how="inner")
predecir["person"].count() - df_persons["person"].count()

0

# Distancia en dias de 1er y ultimo dia de entrada al sitio

In [12]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
primer_dia = timestamp_df.groupby("person")["timestamp"].min().rename("primer").reset_index()
distancia_dias = pd.merge(primer_dia, ultimo_dia, on="person", how="inner")
distancia_dias["distan_dias"] = (distancia_dias["ultimo"] - distancia_dias["primer"]).dt.days
distancia_dias = distancia_dias[["person", "distan_dias"]]
distancia_dias.head()

,person,distan_dias
0,0008ed71,0
1,00091926,27
2,00091a7a,0
3,000ba417,9
4,000c79fe,0


In [13]:
# Veo cuantas personas a predecir pierdo
distancia_dias_predecir = pd.merge(distancia_dias, df_persons, on="person", how="inner")
distancia_dias_predecir["person"].count() - df_persons["person"].count()

0

## Cantidad de modelos distintos vistos por cada usuario

In [14]:
modelos_df = df[["person", "model"]]
cant_modelos = modelos_df.groupby("person")["model"].nunique().rename("modelos_dist").reset_index()
cant_modelos.head()

,person,modelos_dist
0,0008ed71,3
1,00091926,36
2,00091a7a,3
3,000ba417,26
4,000c79fe,1


In [15]:
# Veo cuantas personas a predecir pierdo
cant_modelos_predecir = pd.merge(cant_modelos, df_persons, on="person", how="inner")
cant_modelos_predecir["person"].count() - df_persons["person"].count()

0

## Cantidad de días distintos de entrada al sitio

In [16]:
df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()
cant_dias_dist.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,person,cant_dias_dist
0,0008ed71,1
1,00091926,22
2,00091a7a,1
3,000ba417,3
4,000c79fe,1


In [17]:
# Veo cuantas personas a predecir pierdo
cant_dias_dist_predecir = pd.merge(cant_dias_dist, df_persons, on="person", how="inner")
cant_dias_dist_predecir["person"].count() - df_persons["person"].count()

0

# Entro como 'new' en el mes 5

In [18]:
df_mes_5 = df[["person", "month", "new_vs_returning"]].loc[df["month"] == 5]
df_mes_5["nuevo_mes_5"] = (df_mes_5["new_vs_returning"] == "New") * 1
df_mew_5 = df_mes_5[["person", "nuevo_mes_5"]]
df_mew_5 = df_mew_5.sort_values("nuevo_mes_5", ascending=False)
df_mew_5 = df_mew_5.drop_duplicates(subset=['person'], keep='first')
df_mew_5 = pd.merge(df_mew_5, todas_las_personas, on="person", how="right").fillna(0)
df_mew_5.head()

,person,nuevo_mes_5
0,c0f2beb1,1.0
1,9ff9d5d2,1.0
2,ae9e5fa3,1.0
3,bbbe368d,1.0
4,0b7a3edd,1.0


In [19]:
# Veo cuantas personas a predecir pierdo
df_mew_5_predecir = pd.merge(df_mew_5, df_persons, on="person", how="inner")
df_mew_5_predecir["person"].count() - df_persons["person"].count()

0

# Distancia entre ultima entrada al sitio y el ultimo dia del mes 5 que tenemos

In [20]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia_de_todos = timestamp_df["timestamp"].max()
ultima_entrada = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
ultima_entrada["dias_hasta_ultimo"] = (ultimo_dia_de_todos - ultima_entrada["ultimo"]).dt.days
ultima_entrada = ultima_entrada[["person", "dias_hasta_ultimo"]]
ultima_entrada.head()

,person,dias_hasta_ultimo
0,0008ed71,14
1,00091926,0
2,00091a7a,66
3,000ba417,5
4,000c79fe,2


In [21]:
# Veo cuantas personas a predecir pierdo
df_predecir = pd.merge(ultima_entrada, df_persons, on="person", how="inner")
df_predecir["person"].count() - df_persons["person"].count()

0

## Promedio de entradas por dia

In [22]:
entradas_tot = df.groupby("person").agg({"person": "count"})
#cant_entradas_tot = entradas_tot[["person", "entradas_tot"]]
entradas_tot.columns = ["cant_entradas"]
entradas_tot.reset_index(inplace=True)

df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()

entradas_tot_y_dias = pd.merge(entradas_tot, cant_dias_dist, on="person", how="inner")
entradas_tot_y_dias["promedio_por_dia"] = (entradas_tot_y_dias["cant_entradas"] / entradas_tot_y_dias["cant_dias_dist"])
entradas_tot_y_dias.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,person,cant_entradas,cant_dias_dist,promedio_por_dia
0,0008ed71,6,1,6.000000
1,00091926,448,22,20.363636
2,00091a7a,10,1,10.000000
3,000ba417,206,3,68.666667
4,000c79fe,17,1,17.000000


In [23]:
entradas_tot_y_dias = entradas_tot_y_dias[["person", "promedio_por_dia"]]

In [24]:
# Veo cuantas personas a predecir pierdo
entradas_tot_y_dias_predecir = pd.merge(entradas_tot_y_dias, df_persons, on="person", how="inner")
entradas_tot_y_dias_predecir["person"].count() - df_persons["person"].count()

0

## Promedio de entradas por mes

In [25]:
df_mes = df[["person", "month"]]
cant_meses_dist = df_mes.groupby("person")["month"].nunique().rename("cant_meses_dist").reset_index()

entradas_tot_y_meses = pd.merge(entradas_tot, cant_meses_dist, on="person", how="inner")
entradas_tot_y_meses["promedio_por_mes"] = (entradas_tot_y_meses["cant_entradas"] / entradas_tot_y_meses["cant_meses_dist"])
entradas_tot_y_meses = entradas_tot_y_meses[["person", "promedio_por_mes"]]

entradas_tot_y_meses.head()

,person,promedio_por_mes
0,0008ed71,6.0
1,00091926,448.0
2,00091a7a,10.0
3,000ba417,206.0
4,000c79fe,17.0


In [26]:
# Veo cuantas personas a predecir pierdo
entradas_tot_y_meses_predecir = pd.merge(entradas_tot_y_meses, df_persons, on="person", how="inner")
entradas_tot_y_meses_predecir["person"].count() - df_persons["person"].count()

0

# Mes que entró por primera vez

In [27]:
primer_entrada = df.groupby('person')['timestamp'].min().reset_index()
primer_entrada['mes_primer_entrada'] = primer_entrada['timestamp'].dt.month
primer_entrada = primer_entrada[['person', 'mes_primer_entrada']]

## Promedio de entradas por dia en el mes 5

In [28]:
df_mes_5 = df.loc[df['month'] == 5]
entradas_tot_mes_5 = df_mes_5.groupby("person").agg({"person": "count"})
#cant_entradas_tot = entradas_tot[["person", "entradas_tot"]]
entradas_tot_mes_5.columns = ["cant_entradas"]
entradas_tot_mes_5.reset_index(inplace=True)

df_dias_mes_5 = df_mes_5[["person", "timestamp"]]
df_dias_mes_5["day_of_year"] = df_dias_mes_5["timestamp"].dt.dayofyear
cant_dias_dist_mes_5 = df_dias_mes_5.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()

promedio_dia_entradas_mes_5 = pd.merge(entradas_tot_mes_5, cant_dias_dist_mes_5, on="person", how="inner")
promedio_dia_entradas_mes_5["promedio_por_dia"] = (promedio_dia_entradas_mes_5["cant_entradas"] / promedio_dia_entradas_mes_5["cant_dias_dist"])

# Agrego los que faltan
promedio_dia_entradas_mes_5 = pd.merge(promedio_dia_entradas_mes_5, todas_las_personas, on='person', how='right').fillna(0)

promedio_dia_entradas_mes_5.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,person,cant_entradas,cant_dias_dist,promedio_por_dia
0,0008ed71,6.0,1.0,6.000000
1,00091926,448.0,22.0,20.363636
2,000ba417,206.0,3.0,68.666667
3,000c79fe,17.0,1.0,17.000000
4,000e4d9e,411.0,9.0,45.666667


In [29]:
promedio_dia_entradas_mes_5 = promedio_dia_entradas_mes_5[["person", "promedio_por_dia"]]

# Merge de los features

In [30]:
df_final = pd.merge(cant_por_evento_mes_cinco, ultima_entrada, on='person', how='inner')
df_final = pd.merge(df_final, cant_por_evento_mes_cuatro, on='person', how='inner')
df_final = pd.merge(df_final, distancia_dias, on='person', how='inner')
df_final = pd.merge(df_final, df_mew_5, on='person', how='inner')
df_final = pd.merge(df_final, cant_dias_dist, on='person', how='inner')
df_final = pd.merge(df_final, cant_modelos, on='person', how='inner')
df_final = pd.merge(df_final, entradas_tot_y_dias, on='person', how='inner')
df_final = pd.merge(df_final, entradas_tot_y_meses, on='person', how='inner')
df_final = pd.merge(df_final, primer_entrada, on='person', how='inner')
df_final = pd.merge(df_final, promedio_dia_entradas_mes_5, on='person', how='inner')

df_final.head()

,person,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,lead mes 5,search engine hit mes 5,searched products mes 5,staticpage mes 5,...,viewed product mes 4,visited site mes 4,distan_dias,nuevo_mes_5,cant_dias_dist,modelos_dist,promedio_por_dia_x,promedio_por_mes,mes_primer_entrada,promedio_por_dia_y
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1.0,1,3,6.000000,6.0,5,6.000000
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,27,1.0,22,36,20.363636,448.0,5,20.363636
2,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,...,0.0,0.0,9,1.0,3,26,68.666667,206.0,5,68.666667
3,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,...,0.0,0.0,0,1.0,1,1,17.000000,17.0,5,17.000000
4,000e4d9e,19.0,17.0,1.0,0.0,17.0,0.0,5.0,0.0,0.0,...,0.0,0.0,14,1.0,9,37,45.666667,411.0,5,45.666667


# A csv

In [31]:
# A csv

df_final.to_csv("features_seba.csv", encoding='utf-8', index=False)

---

-----------------------

# Pruebas

In [42]:
# Merge

df_train = pd.merge(df_final, labels, on="person", how="inner")\
            .drop(columns = ['person', 'staticpage mes 5', 'lead mes 5', 'staticpage mes 4', 'lead mes 4'])


In [43]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [44]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [45]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [46]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [47]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=12, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [48]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [49]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9163608267336295
Test acuracy:  0.900592325521504
ROC auc score:  0.8480433920507604
Confusion matrix: 
[[3428  242]
 [ 144   69]]


# pruebas a borrar

# Cantidad de veces que realiza un evento dado

In [51]:
cant_por_evento = pd.concat([pd.get_dummies(df['event']),df[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()

cant_por_evento_final = pd.merge(cant_por_evento, df_final, on='person', how='inner')
df_train = pd.merge(cant_por_evento_final, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product', 'visited site', 'staticpage', 'lead', 'staticpage mes 5', 'lead mes 5', 'staticpage mes 4', 'lead mes 4'])



In [52]:
#veo cuantos users pierdo
cant_por_evento_predecir = pd.merge(cant_por_evento, df_persons, on="person", how="inner")
cant_por_evento_predecir["person"].count() - df_persons["person"].count()

0

In [53]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [54]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [55]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [56]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 10, min_child_weight=1)


In [57]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=10, seed=None,
       silent=True, subsample=1)

In [58]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [59]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8367136694353229
Test acuracy:  0.8318310584599536
ROC auc score:  0.8471472208198851
Confusion matrix: 
[[3102  590]
 [  63  128]]


## Cantidad de eventos en un dia del mes

In [60]:
df_day = pd.concat([df['person'],pd.get_dummies(df['day'])],axis = 1).groupby('person').sum().\
    reset_index()

In [61]:
columnas = ['dia_' + str(i) for i in range(1,32)]
columnas.insert(0,'person')
df_day.columns = columnas

In [62]:
df_con_dias_del_mes_final = pd.merge(cant_por_evento_final, df_day, on='person', how='inner')

df_train = pd.merge(df_con_dias_del_mes_final, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [63]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [64]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [65]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [66]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=17)


In [67]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [68]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [69]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.897430944562488
Test acuracy:  0.8900334792686068
ROC auc score:  0.8409522783094053
Confusion matrix: 
[[3369  323]
 [ 104   87]]


## Cantidad de eventos en top K semanas del año.

In [70]:
K = 10
df['week_of_year'] = df['timestamp'].dt.weekofyear

In [71]:
df_week_of_year = df[['person','week_of_year']]

top_K_week_of_year = df_week_of_year['week_of_year'].value_counts().head(K).index

df_week_of_year = pd.concat([df_week_of_year,pd.get_dummies(df_week_of_year['week_of_year'])[top_K_week_of_year]],axis = 1)

df_week_of_year.drop(columns = ['week_of_year'],inplace = True)

df_week_of_year1 = df_week_of_year.groupby('person').sum().reset_index()

In [72]:
df_con_top_semanas_final = pd.merge(df_con_dias_del_mes_final, df_week_of_year1, on='person', how='inner')

df_train = pd.merge(df_con_top_semanas_final, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [73]:
df_train.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,search engine hit,searched products,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,...,20,22,19,18,17,16,15,14,13,label
0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.0,0.0,1.0,0.0,1.0,1.0,9.0,1.0,0.0,1.0,...,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,5.0,0.0,1.0,0.0,4.0,0.0,4.0,5.0,0.0,1.0,...,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,29.0,165.0,15.0,2.0,28.0,13.0,11.0,16.0,113.0,4.0,...,181.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [74]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [75]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [76]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [77]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=17)


In [78]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [79]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [81]:
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, proba)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8969802330822226
Test acuracy:  0.8954416688127737

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.92      0.94      3692
           1       0.23      0.48      0.31       191

   micro avg       0.90      0.90      0.90      3883
   macro avg       0.60      0.70      0.63      3883
weighted avg       0.93      0.90      0.91      3883


ROC auc score:  0.8478271967690153
Confusion matrix: 
[[3386  306]
 [ 100   91]]


## Visitas a la pagina segun resolucion

In [82]:
df_resolucion = df.loc[df['event'] == 'visited site']
df_resolucion = df_resolucion[['person','screen_resolution']].dropna()

top_5_resoluciones = df_resolucion['screen_resolution'].value_counts().head(10).index
df_resolucion = df_resolucion.loc[df_resolucion['screen_resolution'].isin(top_5_resoluciones)]
df_resolucion = pd.concat([df_resolucion['person'],\
        pd.get_dummies(df_resolucion['screen_resolution'])],axis = 1).groupby('person').sum().reset_index()

df_resolucion = pd.merge(df_resolucion, todas_las_personas, on="person", how="right").fillna(0)

df_resolucion.head()

,person,1024x768,1360x768,1366x768,1440x900,1600x900,1920x1080,320x534,320x568,320x570,360x640
0,0008ed71,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,00091926,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,000ba417,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [83]:
# Creo que es mucho 20 resoluciones, probar (quizas empeore)..

In [84]:
# Veo cuantas personas a predecir pierdo
df_resolucion_predecir = pd.merge(df_resolucion, df_persons, on="person", how="inner")
df_resolucion_predecir["person"].count() - df_persons["person"].count()

0

In [85]:
# Merge

df_con_resoluciones_final = pd.merge(df_con_top_semanas_final, df_resolucion, on="person", how="inner") 
df_train = pd.merge(df_con_resoluciones_final, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])


In [86]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [87]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [88]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [89]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [90]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=12, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [91]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [92]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9123044234112421
Test acuracy:  0.9078032449137265
ROC auc score:  0.8506931642209276
Confusion matrix: 
[[3443  249]
 [ 109   82]]


# Distancia entre ultima entrada al sitio y el ultimo dia que tenemos

In [148]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia_de_todos = timestamp_df["timestamp"].max()
ultima_entrada = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
ultima_entrada["dias_hasta_ultimo"] = (ultimo_dia_de_todos - ultima_entrada["ultimo"]).dt.days
ultima_entrada = ultima_entrada[["person", "dias_hasta_ultimo"]]
ultima_entrada.head()

,person,dias_hasta_ultimo
0,0008ed71,14
1,00091926,0
2,00091a7a,66
3,000ba417,5
4,000c79fe,2


In [149]:
# Veo cuantas personas a predecir pierdo
df_predecir = pd.merge(ultima_entrada, df_persons, on="person", how="inner")
df_predecir["person"].count() - df_persons["person"].count()

0

In [150]:
# Merge
# df_con_resoluciones

df_con_diatancia_hasta_ultimo_final = pd.merge(df_con_resoluciones_final, ultima_entrada, on="person", how="inner") 
df_train = pd.merge(df_con_diatancia_hasta_ultimo_final, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])


In [151]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [152]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [153]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [154]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [155]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=12, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [156]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [157]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9233146610005795
Test acuracy:  0.9147566314705124
ROC auc score:  0.8535952930632527
Confusion matrix: 
[[3480  212]
 [ 119   72]]


In [158]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [124]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [159]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [126]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [141]:
from sklearn.feature_selection import RFE

# create the RFE model and select n attributes
rfe = RFE(xg_reg, 21)
rfe = rfe.fit(X, y)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False  True  True  True False  True  True False  True  True False False
  True False  True False  True False False False False False False False
 False False False False  True False  True  True  True  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False  True False  True False False  True False False
 False False False False False  True False  True False False False False
 False False False False]
[61  1  1  1  8  1  1 11  1  1  3 12  1 18  1 45  1 38 63 31 58 15 65 26
  6 35 39 36  1 55  1  1  1  1 28  9 52 33 54 46 48 50 66 68 59 57 53 51
 62 49 47 44 43 10  4 21 14 27 32 25  5 17  1 24  1 22  1 16 23  1  2  7
 60 64 56 34 37  1 13  1 42 40 19 29 30 41 20 67]


In [135]:
mask = rfe.get_support()
features_X = X.columns[mask]
new_x = X.filter(items=features_X)
new_x.head()

,conversion,search engine hit,brand listing mes 5,checkout mes 5,search engine hit mes 5,viewed product mes 5,dias_hasta_ultimo_x,distan_dias,modelos_dist,promedio_por_dia_x,promedio_por_mes,dia_31,22,1024x768,1366x768
0,0.0,0.0,0.0,3.0,0.0,0.0,14,0,3,6.000000,6.00,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,3.0,2,0,1,17.000000,17.00,0.0,17.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,4.0,0,0,2,19.000000,19.00,19.0,19.0,0.0,0.0
3,2.0,13.0,113.0,4.0,7.0,123.0,9,114,26,33.642857,117.75,0.0,0.0,0.0,19.0
4,1.0,0.0,1.0,2.0,0.0,2.0,9,0,1,7.000000,7.00,0.0,0.0,0.0,0.0


In [136]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(new_x, y, test_size=0.2, random_state=123)

In [137]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [138]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=12, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [139]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [140]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9336810250466808
Test acuracy:  0.9327839299510687
ROC auc score:  0.8487673929197417
Confusion matrix: 
[[3578  114]
 [ 147   44]]


# Pruebo el RFE iterando

In [164]:
for n in range(10,35,5):
    # create the RFE model and select n attributes
    rfe = RFE(xg_reg, n)
    rfe = rfe.fit(X, y)
    
    print('n = ', n)
    
    # Selecciono los n features para entrenar
    mask = rfe.get_support()
    features_X = X.columns[mask]
    new_x = X.filter(items=features_X)
    
    # Entrenamos y predecimos
    X_train, X_test, y_train, y_test = train_test_split(new_x, y, test_size=0.2, random_state=123)
    
    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)

    xg_reg.fit(X_train,y_train)
    proba = xg_reg.predict_proba(X_test)[:,1]
    print("ROC auc score: ", roc_auc_score(y_test, proba))


n =  10
ROC auc score:  0.8441500796968682
n =  15
ROC auc score:  0.8487673929197417
n =  20
ROC auc score:  0.8526090088659222
n =  25
ROC auc score:  0.8524282019138593
n =  30
ROC auc score:  0.8472932844752769


In [165]:
for n in range(15,35,2):
    # create the RFE model and select n attributes
    rfe = RFE(xg_reg, n)
    rfe = rfe.fit(X, y)
    # summarize the selection of the attributes
    print('n = ', n)
    
    # Selecciono los n features para entrenar
    mask = rfe.get_support()
    features_X = X.columns[mask]
    new_x = X.filter(items=features_X)
    
    # Entrenamos y predecimos
    X_train, X_test, y_train, y_test = train_test_split(new_x, y, test_size=0.2, random_state=123)
    
    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)

    xg_reg.fit(X_train,y_train)
    proba = xg_reg.predict_proba(X_test)[:,1]
    print("ROC auc score: ", roc_auc_score(y_test, proba))

n =  15
ROC auc score:  0.8487673929197417
n =  17
ROC auc score:  0.8540263935607199
n =  19
ROC auc score:  0.8495126011809885
n =  21
ROC auc score:  0.850215975676856
n =  23
ROC auc score:  0.8501479071772561
n =  25
ROC auc score:  0.8524282019138593
n =  27
ROC auc score:  0.8521871259777757
n =  29
ROC auc score:  0.8499288116941682
n =  31
ROC auc score:  0.8481590307045657
n =  33
ROC auc score:  0.8488099357319916


# Pruebo SelectFromModel

igual no se si entendi bien como funciona

In [144]:
from sklearn.feature_selection import SelectFromModel
selection = SelectFromModel(xg_reg, prefit=True)
select_X_train = selection.transform(X_train)
# train model
selection_model = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)

selection_model.fit(select_X_train, y_train)
# eval model
select_X_test = selection.transform(X_test)
preds = selection_model.predict(select_X_test)
proba = selection_model.predict_proba(select_X_test)[:,1]

# Vemos que onda

train_accuracy = accuracy_score(y_train, selection_model.predict(select_X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.936964780117185
Test acuracy:  0.935359258305434
ROC auc score:  0.7456400707912396
Confusion matrix: 
[[3612   80]
 [ 171   20]]


In [160]:
# Fit model using each importance as a threshold
from numpy import sort
thresholds = sort(xg_reg.feature_importances_)
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(xg_reg, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    # train model
    selection_model = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)
    
    selection_model.fit(select_X_train, y_train)
    
    # eval model
    select_X_test = selection.transform(X_test)
    preds = selection_model.predict(select_X_test)
    proba = selection_model.predict_proba(select_X_test)[:,1]
    accuracy = accuracy_score(y_test, preds)
    
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    area_debajo_de_curva = roc_auc_score(y_test, proba)
    print("ROC auc score: ", area_debajo_de_curva)


Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.8535952930632527
Thresh=0.000, n=88, Accuracy: 91.48%
ROC auc score:  0.853595293